# 앞서 만들었던 크롤링/토크나이즈 코드를 통해 만든 딥러닝 모델의 정확도가 처참할 정도로 낮아(ㅜㅜ), 앞선 코드의 단점을 고치고 더 정확도가 높은 챗봇 모델을 만들고자 이 파일을 생성했다.


### 0.출저: 크롤링을 한 사이트 목록이다.

1.세종대왕의 업적과 외교:https://theonlyone.tistory.com/entry/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95%EC%9D%98-%EC%97%85%EC%A0%81%EA%B3%BC-%EC%99%B8%EA%B5%90#:~:text=%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95%C2%A0%EC%8B%9C%EB%8C%80%EC%9D%98%C2%A0%EC%99%B8%EA%B5%90%EA%B4%80%EA%B3%84%EB%8A%94%C2%A0%EB%8B%A4%EC%96%91%ED%95%9C%C2%A0%EC%9A%94%EC%9D%B8%EC%97%90%C2%A0%EB%94%B0%EB%9D%BC%C2%A0%EB%B3%80%ED%99%94%ED%96%88%EC%A7%80%EB%A7%8C%2C%C2%A0%EB%8C%80%EC%B2%B4%EB%A1%9C%C2%A0%EC%95%88%EC%A0%95%EC%A0%81%EC%9D%B4%EA%B3%A0%C2%A0%ED%99%9C%EB%B0%9C%ED%95%9C%C2%A0%EC%99%B8%EA%B5%90%C2%A0%ED%99%9C%EB%8F%99%EC%9D%B4%C2%A0%EC%9D%B4%EB%A3%A8%EC%96%B4%EC%A1%8C%EC%8A%B5%EB%8B%88%EB%8B%A4,%EC%A0%95%EC%B9%98%EC%A0%81%EC%9D%B8%C2%A0%EC%95%88%EC%A0%95%3A%C2%A0%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95%EC%9D%98%C2%A0%ED%86%B5%EC%B9%98%EB%8A%94%C2%A0%EC%9D%BC%EC%A0%95%ED%95%9C%C2%A0%EC%A0%95%EC%B9%98%EC%A0%81%EC%9D%B8%C2%A0%EC%95%88
2.세종-한국민족문화대백과사전: https://encykorea.aks.ac.kr/Article/E0029857

### 1. import 하기

코딩을 하는데 필요한 것들을 모두 import 한다.
한 곳에 모아서 import 하는 이유는, 그래야 한눈에 보기도 편하고 관리도 편하기 때문이다.


In [1]:
import requests
from bs4 import BeautifulSoup
from urllib import request as req #url을 통해 정보를 불러옴.
import re #파이썬에서 정규 표현식을 사용할 때, re을 쓴다.
import nltk #자연어 처리 모델인 nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from konlpy.tag import Okt

#본격적인 딥러닝 등을 위한 import 문이다.
import numpy as np
import tensorflow as tf #오픈 소스 기계학습 라이브러리
from tensorflow.keras.preprocessing.sequence import pad_sequences 
#pad_sequences 는 문장 등을 패딩해서 길이를 맞추는 데에 사용된다. 텍스트 데이터 처리를 할 때 사용된다.

from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense, Dropout, Add
from tensorflow.keras.layers import Input, LayerNormalization, Dropout, Dense, Attention
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, Attention,Input
from tensorflow.keras.layers import Reshape
#GRU는 LSTM과 비슷한 순환 신경망 구조이다. 시퀀스 데이터를 처리하고 정보를 장기적으로 유지하는 데 사용된다.
#Bidirectional 레이어는 양방향 순환 신경망을 구현한다. 입력 시퀀스를 순방향/역방향으로 처리해서 학습 효과를 높인다.
#Attention레이어는 입력 시퀀스의 각 단어에 가중치를 할당해서 핵심 정보에 집중할 수 있도록 돕는 레이어이다.
from tensorflow.keras.models import Sequential
#Sequential은 keras에서 신경망 모델을 구축하는 데 가장 간단한 방법 중 하나이다. 순차적으로 레이어를 쌓아서 모델을 만들 수 있다.
#keras는 파이썬으로 작성된 오픈 소스 신경망 라이브러리이다.

from keras.layers import Dense #완전 연결된 레이어를 만드는 데 사용된다. 이 레이어는 신경망의 중간 또는 출력 레이어로 사용된다.
from keras.models import Model #케라스의 모델을 정의하는 데 사용된다. 모델을 사용해서 입력과 출력을 정의한 모델을 생성할 수 있다.
from tensorflow.keras import activations
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences #tensorflow의 keras 패키지에서 sequence(대용량의 데이터도 효율적으로 학습 가능) 데이터를 전처리 하기 위한 pad_sequences 함수를 가져온다.
from tensorflow.keras.callbacks import ModelCheckpoint #모델 훈련 중에 모델의 상태를 저장하기 위한 콜백함수이다. 중간에 저장하면 최상의 성능을 보인 모델을 선택할 수 있다.
from tensorflow.keras.models import load_model #디스크에 저장된 훈련된 모델을 메모리로 로드할 수 있다.
from sklearn.model_selection import train_test_split #데이터를 훈련 세트와 테스트 세트로 나누기 위한 함수이다.
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dense, Dropout, Add
from keras.callbacks import EarlyStopping #모델 학습을 조기 종료하는 데 쓰인다.
from tensorflow.keras import backend as K



#시각화를 위한 import 이다.
import matplotlib.pyplot as plt

from konlpy.tag import Mecab

## 2.데이터 불러오기,크롤링 하기 


url을 적어서 데이터를 불러온 다음 soup을 이용해서 자료를 긁는다.


In [2]:
url='https://theonlyone.tistory.com/entry/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95%EC%9D%98-%EC%97%85%EC%A0%81%EA%B3%BC-%EC%99%B8%EA%B5%90#:~:text=%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95%C2%A0%EC%8B%9C%EB%8C%80%EC%9D%98%C2%A0%EC%99%B8%EA%B5%90%EA%B4%80%EA%B3%84%EB%8A%94%C2%A0%EB%8B%A4%EC%96%91%ED%95%9C%C2%A0%EC%9A%94%EC%9D%B8%EC%97%90%C2%A0%EB%94%B0%EB%9D%BC%C2%A0%EB%B3%80%ED%99%94%ED%96%88%EC%A7%80%EB%A7%8C%2C%C2%A0%EB%8C%80%EC%B2%B4%EB%A1%9C%C2%A0%EC%95%88%EC%A0%95%EC%A0%81%EC%9D%B4%EA%B3%A0%C2%A0%ED%99%9C%EB%B0%9C%ED%95%9C%C2%A0%EC%99%B8%EA%B5%90%C2%A0%ED%99%9C%EB%8F%99%EC%9D%B4%C2%A0%EC%9D%B4%EB%A3%A8%EC%96%B4%EC%A1%8C%EC%8A%B5%EB%8B%88%EB%8B%A4,%EC%A0%95%EC%B9%98%EC%A0%81%EC%9D%B8%C2%A0%EC%95%88%EC%A0%95%3A%C2%A0%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95%EC%9D%98%C2%A0%ED%86%B5%EC%B9%98%EB%8A%94%C2%A0%EC%9D%BC%EC%A0%95%ED%95%9C%C2%A0%EC%A0%95%EC%B9%98%EC%A0%81%EC%9D%B8%C2%A0%EC%95%88'
response = requests.get(url)
response.encoding = 'utf-8' #utf-8로 특정화한다.

#BeautifulSoup을 사용하여 HTML 파싱
soup=BeautifulSoup(response.text,'html.parser')

# 텍스트 추출
text = soup.get_text()

# 결과 출력
print(text)














































































세종대왕의 업적과 외교




















본문 바로가기






              
              
                the only one
              
            



블로그 내 검색

검색





관리
글쓰기
로그인
로그아웃



메뉴



홈
태그
방명록










역사
세종대왕의 업적과 외교

by 뮤콥
2024. 1. 30.













 
세종대왕은 조선시대 한국의 제4대 군주로, 조선의 중추적인 왕 중 하나로 인정받고 있습니다. 그의 통치는 한국 역사상 가장 영향력 있는 것 중 하나로 여겨지며, 세종 대왕은 문화, 과학, 정치, 교육 등 여러 분야에서 혁신적인 업적을 이루었습니다. 한글 창제: 세종 대왕은 한글을 창제하여 한글 문자를 만들었습니다. 이는 한국의 문자 체계를 혁명적으로 변화시켜, 많은 사람들에게 글쓰기와 읽기를 쉽게 접근할 수 있게 했습니다. 과학기술 발전: 세종 대왕은 조선의 과학기술 발전을 촉진하기 위해 노력했습니다. 천문학, 의학, 농업 등 다양한 분야에서 연구를 진행하고 기술을 발전시켰습니다. 정치혁신: 세종 대왕은 조선의 행정 체계를 혁신하여 국가의 효율성을 높였습니다. 특히 세종 대왕은 양인을 인정하고, 서민들의 권리를 보호하는 등 사회적으로 진보적인 정책을 시행했습니다. 교육정책: 세종 대왕은 교육을 중요시하여 사대부의 특권을 제한하고 서민들의 교육을 촉진했습니다. 또한 신진 문인들을 후원하여 문화 예술의 발전에 기여했습니다. 그의 통치는 한국의 역사와 문화에 큰 영향을 끼쳤습니다. 그의 업적은 현재도 한국 사회에서 크게 존경받고 있으며, 그의 이름은 한국의 국민들에게 특별한 의미를 지니고 있습니다. 세종대왕(1397년 ~ 1450년)은 조선시대 한국의 제4대 군주로, 그의 유년기에 대한 자세

In [3]:
url='https://encykorea.aks.ac.kr/Article/E0029857'
response = requests.get(url)
response.encoding = 'utf-8' #utf-8로 특정화한다.

#BeautifulSoup을 사용하여 HTML 파싱
soup=BeautifulSoup(response.text,'html.parser')

# 텍스트 추출
text1 = soup.get_text()

# 결과 출력
print(text1)


















세종(世宗) - 한국민족문화대백과사전

























본문 바로가기







의견&문의
이용 안내



































분야



경제·산업

경제
교통
산업
통신



과학

과학기술
동물
식물
의약학



교육


문학

고전산문
고전시가
구비문학
한문학
현대문학



사회

가족
사회구조
촌락



생활

민속·인류
식생활
의생활
주생활



언론·출판

언론·방송
출판



언어

언어·문자



역사

선사문화
고대사
고려시대사
조선시대사
근대사
현대사



예술·체육

건축
공예
국악
대중음악
무용
서예
연극
영화
조각
체육
현대음악
회화



정치·법제

국방
법제·행정
외교
정치



종교·철학

개신교
대종교
도교
민간신앙
불교
신종교
원불교
유교
천도교
천주교



지리

인문지리
자연지리



북한




유형


인물


지명


개념


유물


단체


작품


문헌


사건


물품


제도


놀이


유적


의식/행사


동/식물





전체메뉴




















세종

(世宗)













목차


메뉴 접기




정의
개설
세종의 즉위 과정
세종의 업적

1. 유교정치의 기틀 마련
2. 편찬사업의 융성
3. 훈민정음의 창제
4. 과학기술의 발전과 기술서적의 편찬
5. 법전의 정비
6. 불교에 대한 시책


세종 치세의 역사적 의의
상훈과 추모
참고문헌


관련 정보


관련 항목
관련 미디어













글자 크기





의견 제시






URL 복사



페이스북 공유



트위터 공유







						세종
(世宗)					









									세종대왕 동상
								





조선시대사

인물


조선의 제4대(재위: 1418년~1450년) 왕.




이칭


자
원정(元正)


시호
장헌(莊憲)



인물/전통 인물


성별

남성							

### 3.데이터를 다듬어서 txt 파일로 저장한 것을 불러오기 

위의 크롤링 코드를 보면 알겠지만, 비문도 많고, 목차, 블로그 자체의 매뉴 등 챗봇을 만드는 데에 방해되는 요소가 아주 많다.
그렇기 때문에 위의 데이터를 다듬어서 txt로 저장한 다음, 이를 불러오는 식으로 데이터를 처리하자.


In [7]:
#파일 읽기 함수

def read_file(file_path):
    with open(file_path,"r",encoding="utf-8") as file:
        return file.read()

#2개의 파일 경로
file1="세종-한국민족문화대백과사전.txt"
file2="세종대왕의 업적과 외교.txt"

#파일 내용 읽기

data1 = read_file(file1)
data2 = read_file(file2)

#2개의 파일을 하나로 합친다.
total_data = data1 + "\n" + data2

#출력을 통해 2개의 텍스트 파일이 잘 합쳐졌는지 확인한다.
print(total_data)



세종의 즉위 과정:원래 태종의 뒤를 이을 왕세자는 양녕대군(讓寧大君)이었다. 그러나 양녕대군이 개와 매[鷹]에 관계된 사건을 비롯해, 세자로서의 품위를 손상시킨 일련의 행동과 사건들로 인해 태종의 선위에 대한 마음이 동요되었다. 그래서 태종은 자신이 애써 이룩한 정치적 안정과 왕권을 이어받아 훌륭한 정치를 펴기에 양녕대군이 적합하지 못하다고 판단하였다. 태종의 마음이 이미 세자 양녕대군에게서 떠난 것을 알게 된 신료(臣僚)들은 그를 폐위할 것을 청하는 소(疏)를 올려 양녕대군을 폐하고 충녕대군을 왕세자로 삼기에 이르렀다.

이 때 태종에게는 왕후 민씨 소생으로 양녕 · 효령(孝寧) · 충녕 등 세 대군이 있었고, 양녕대군에게도 두 아들이 있었다. 따라서 그를 폐하고 새로이 세자를 세우는 일은 매우 어려운 일이었기 때문에 세자 폐립에 관해 의론이 분분하였다. 그러나 태종의 마음은 이미 셋째아들인 충녕대군에게 쏠려 있었다. 1418년 6월에 태종은 “충녕대군은 천성이 총민하고, 또 학문에 독실하며 정치하는 방법 등도 잘 안다.”라고 해 택현(擇賢)주17의 명분을 주어 세자로 책봉하기로 결정하였다.

이처럼 충녕대군에 대한 세자책봉은 태종의 뜻에 따라 극적으로 이루어졌다. 물론, 대부분의 신하들도 이를 환영하였다. 두 달 뒤인 1418년 8월 10일 태종의 선위를 이어받아 세자 충녕대군이 왕위에 올랐으니 이 사람이 세종이다.



유교정치의 기틀 마련:세종대는 우리 민족의 역사에서 가장 훌륭한 유교정치, 찬란한 문화가 이룩된 시대였다. 이 시기에는 정치적으로 안정되어 정치 · 경제 · 사회 · 문화 등 전반적인 기틀을 잡은 시기였다. 즉, 집현전을 통해 많은 인재가 양성되었고, 유교정치의 기반이 되는 의례 · 제도가 정비되었으며, 다양하고 방대한 편찬 사업이 이루어졌다. 또한 농업과 과학기술의 발전, 의약기술과 음악 및 법제의 정리, 공법(貢法)의 제정, 국토의 확장 등 수많은 사업을 통해 민족국가의 기틀을 확고히 하였다. 이 많은 일들을 주도한 인물이 바로 세

### 4.nltk을 통한 토큰화,konlpy를 이용한 형태소 분석

nltk(natural language toolkit)은 자연어 처리를 위한 파이썬 패키지이다. 아나콘다를 설치했다면 nltk는 기본적으로 설치가 되어 있으므로 안심하시라.

konlpy는 한글 텍스트를 형태소 단위로 분석하게 해주는 파이썬 라이브러리이다.

In [6]:
#nltk 문장 토큰화.

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\djwls\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
#nltk을 통해 문장을 토큰화한다.
sentences = nltk.sent_tokenize(total_data)

In [9]:
#konlpy를 이용한 형태소 분석

okt = Okt()
tokenized_sentences = [okt.morphs(sentence) for sentence in sentences]

#토큰화된 결과 저장

# 토큰화된 결과 저장
with open('tokenized_sentences.txt', 'w', encoding='utf-8') as file:
    for sentence in tokenized_sentences:
        file.write(' '.join(sentence) + '\n')

# 토큰화된 문장 로드
with open('tokenized_sentences.txt', 'r', encoding='utf-8') as file:
    tokenized_sentences = file.readlines()

tokenized_sentences = [sentence.strip() for sentence in tokenized_sentences]


### 5. 토큰화, 정수 인코딩


In [10]:
#토큰화 및 정수 인코딩
#keras의 함수중 하나인 tokenizer를 통해 토크나이징을 한다. 
#토크나이저는 nlp의 핵심 중 하나로, 입력된 텍스트를 모델에서 처리할 수 있는 데이터로 변환하는 것이다.

tokenizer = Tokenizer() #토크나이저를 정의한다.
tokenizer.fit_on_texts(tokenized_sentences) #앞서 처리한 데이터를 토크나이징에 초대한다.
words_len = len(tokenizer.word_index)+1 #신경망 모델에서 사용하는 단어 인덱스가 1부터 시작하기 때문에 1을 더한다. 사실 상식적으로는 이게 더 common sence인데 컴공의 인덱싱이 약간 이상한 것이다.



